In [1]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, BertModel, BertForSequenceClassification, BertTokenizer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Concatenate, GlobalAveragePooling1D
import torch
from torchsummary import summary
import tensorflow as tf
import datasets
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn import metrics
from time import time
import gc

## FRENK Dataset

In [2]:
def truncate(texts,max_length):
    for i in range(len(texts)):
      if len(texts[i]) > max_length:
        texts[i] = texts[i][:max_length]
    return texts

In [3]:
#FRENK dataset
ds = datasets.load_dataset("classla/FRENK-hate-en","binary")
frenk_train = ds["train"]["text"]
frenk_label_train = np.array(ds["train"]["label"])
frenk_test = ds["test"]["text"]
frenk_label_test = np.array(ds["test"]["label"])

In [4]:
max_length = 150
frenk_train = truncate(frenk_train,max_length)
frenk_test = truncate(frenk_test,max_length)

In [5]:
print(np.array(frenk_test).shape)

(2301,)


## Gab Dataset

In [6]:
def get_texts(df):
    np_array = df["text"].to_numpy()
    texts = []
    for i in range(len(np_array)):
        texts.append(np_array[i])
    return texts

In [7]:
def get_hate_labels(labels):
    hate_labels = np.zeros(len(labels))
    for i in range(len(labels)):
        if labels[i].any():
            hate_labels[i] = 1
    return hate_labels

In [8]:
#Gab dataset
df_train = pd.read_csv('data/ghc_train.tsv',sep='\t')
ghc_train = get_texts(df_train)
labels = df_train[["hd","cv","vo"]].to_numpy()
ghc_label_train = get_hate_labels(labels)

df_test = pd.read_csv('data/ghc_test.tsv',sep='\t')
ghc_test = get_texts(df_test)
labels = df_test[["hd","cv","vo"]].to_numpy()
ghc_label_test = get_hate_labels(labels)

In [9]:
max_length = 150
ghc_train = truncate(ghc_train,max_length)
ghc_test = truncate(ghc_test,max_length)

## Twi-Red-You dataset

In [10]:
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
df = dataset['train'].to_pandas()
df_hate = df[['text','hate_speech_score']]
df_hate['hate'] = (df_hate['hate_speech_score']>=0.5).astype(int)
df_hate = df_hate[['text','hate']]

df_hate_train = df_hate.iloc[0:8404]
df_hate_test = df_hate.iloc[8404:8404+2301]

df_hate_train.to_csv('data/try_train_8404.tsv', sep="\t", index=False)
df_hate_test.to_csv('data/try_test_2301.tsv', sep="\t", index=False)

Found cached dataset parquet (C:/Users/Pedro/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Pedro\AppData\Local\Temp\ipykernel_5060\1110734405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hate['hate'] = (df_hate['hate_speech_score']>=0.5).astype(int)


In [10]:
#Twi-Red-You dataset
df_train = pd.read_csv('data/try_train_8404.tsv',sep='\t')
try_train = get_texts(df_train)
try_label_train = df_train['hate'].to_numpy()

df_test = pd.read_csv('data/try_test_2301.tsv',sep='\t')
try_test = get_texts(df_test)
try_label_test = df_test['hate'].to_numpy()

In [11]:
max_length = 150
try_train = truncate(try_train,max_length)
try_test = truncate(try_test,max_length)

## Load models

In [12]:
#roBERTa models
sent_MOD = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
aggr_MOD = f"cardiffnlp/twitter-roberta-base-offensive"
gen_MOD = f"roberta-base"

#Load tokenizer and models
tokenizer = AutoTokenizer.from_pretrained(sent_MOD)
sent_mod = TFAutoModel.from_pretrained(sent_MOD)
aggr_mod = TFAutoModel.from_pretrained(aggr_MOD)
gen_mod = TFAutoModel.from_pretrained(gen_MOD)

batch_size = 32

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel

In [13]:
def Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out_1 = []
  batch_out_2 = []
  batch_out_3 = []
  for batch in batches:
      encoded_inputs = tokenizer(batch,padding=True,return_tensors='tf')
      # Compute the outputs of the modules as numpy arrays.
      emb_1 = sent_mod(encoded_inputs)[0]
      pool_1 = GlobalAveragePooling1D()(emb_1).numpy()
      batch_out_1.append(pool_1)

      emb_2 = aggr_mod(encoded_inputs)[0]
      pool_2 = GlobalAveragePooling1D()(emb_2).numpy()
      batch_out_2.append(pool_2)

      emb_3 = gen_mod(encoded_inputs)[0]
      pool_3 = GlobalAveragePooling1D()(emb_3).numpy()
      batch_out_3.append(pool_3)
    
      encoded_inputs = None
      emb_1 = None
      emb_2 = None
      emb_3 = None
      gc.collect() 
    
  outs_1 = np.concatenate(batch_out_1,axis=0)
  outs_2 = np.concatenate(batch_out_2,axis=0)
  outs_3 = np.concatenate(batch_out_3,axis=0)  

  return outs_1,outs_2,outs_3

## Compute outputs (If the outputs are already saved you can skip this step)

### FRENK dataset

In [15]:
#### Train set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,frenk_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/frenk/sent_train_outs.npy',outs_sent)
np.save('outs/frenk/aggr_train_outs.npy',outs_aggr)
np.save('outs/frenk/gen_train_outs.npy',outs_gen)

Elapsed time: 205.9961051940918


In [16]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,frenk_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/frenk/sent_test_outs.npy',outs_sent)
np.save('outs/frenk/aggr_test_outs.npy',outs_aggr)
np.save('outs/frenk/gen_test_outs.npy',outs_gen)

Elapsed time: 71.29464769363403


### Gab dataset

In [17]:
#### Train set
batch_size = 32

t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,ghc_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/ghc/sent_train_outs.npy',outs_sent)
np.save('outs/ghc/aggr_train_outs.npy',outs_aggr)
np.save('outs/ghc/gen_train_outs.npy',outs_gen)

Elapsed time: 509.5066497325897


In [18]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,ghc_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/ghc/sent_test_outs.npy',outs_sent)
np.save('outs/ghc/aggr_test_outs.npy',outs_aggr)
np.save('outs/ghc/gen_test_outs.npy',outs_gen)

Elapsed time: 137.6030786037445


### Twi-Red-You dataset

In [19]:
#### Train set
batch_size = 32

t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,try_train,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/try/sent_train_outs.npy',outs_sent)
np.save('outs/try/aggr_train_outs.npy',outs_aggr)
np.save('outs/try/gen_train_outs.npy',outs_gen)

Elapsed time: 167.61855840682983


In [20]:
#### Test set
t = time()
outs_sent,outs_aggr,outs_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,try_test,batch_size)
print("Elapsed time:",time()-t)

np.save('outs/try/sent_test_outs.npy',outs_sent)
np.save('outs/try/aggr_test_outs.npy',outs_aggr)
np.save('outs/try/gen_test_outs.npy',outs_gen)

Elapsed time: 43.6887104511261


## Load outputs

In [14]:
frenk_sent_train = np.load('outs/frenk/sent_train_outs.npy')
frenk_aggr_train = np.load('outs/frenk/aggr_train_outs.npy')
frenk_gen_train = np.load('outs/frenk/gen_train_outs.npy')

frenk_sent_test = np.load('outs/frenk/sent_test_outs.npy')
frenk_aggr_test = np.load('outs/frenk/aggr_test_outs.npy')
frenk_gen_test = np.load('outs/frenk/gen_test_outs.npy')

In [15]:
ghc_sent_train = np.load('outs/ghc/sent_train_outs.npy')
ghc_aggr_train = np.load('outs/ghc/aggr_train_outs.npy')
ghc_gen_train = np.load('outs/ghc/gen_train_outs.npy')

ghc_sent_test = np.load('outs/ghc/sent_test_outs.npy')
ghc_aggr_test = np.load('outs/ghc/aggr_test_outs.npy')
ghc_gen_test = np.load('outs/ghc/gen_test_outs.npy')

In [16]:
try_sent_train = np.load('outs/try/sent_train_outs.npy')
try_aggr_train = np.load('outs/try/aggr_train_outs.npy')
try_gen_train = np.load('outs/try/gen_train_outs.npy')

try_sent_test = np.load('outs/try/sent_test_outs.npy')
try_aggr_test = np.load('outs/try/aggr_test_outs.npy')
try_gen_test = np.load('outs/try/gen_test_outs.npy')

In [17]:
print(try_sent_train.shape)

(8404, 768)


## Hate prediction with the 3 models

In [18]:
conc_frenk_train = np.concatenate([frenk_sent_train, frenk_aggr_train, frenk_gen_train], axis=1)
conc_frenk_test = np.concatenate([frenk_sent_test, frenk_aggr_test, frenk_gen_test], axis=1)

In [19]:
conc_ghc_train = np.concatenate([ghc_sent_train, ghc_aggr_train, ghc_gen_train], axis=1)
conc_ghc_test = np.concatenate([ghc_sent_test, ghc_aggr_test, ghc_gen_test], axis=1)

In [20]:
conc_ghc_train = conc_ghc_train[:8404]
conc_ghc_test = conc_ghc_test[:2301]

ghc_gen_train = ghc_gen_train[:8404]
ghc_gen_test = ghc_gen_test[:2301]

ghc_label_train = ghc_label_train[:8404]
ghc_label_test = ghc_label_test[:2301]

In [21]:
conc_try_train = np.concatenate([try_sent_train, try_aggr_train, try_gen_train], axis=1)
conc_try_test = np.concatenate([try_sent_test, try_aggr_test, try_gen_test], axis=1)

In [22]:
print(conc_try_train.shape)

(8404, 2304)


In [23]:
##Counterfactual evaluation
def evaluate_text_PEACE(text,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    out_sent,out_aggr,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,text,batch_size)
    conc = np.concatenate([out_sent,out_aggr,out_gen],axis=1)
    prob = model.predict(conc)
    print("Hate prob.:",prob)

In [24]:
n_avg = 1

### Training with FRENK

In [25]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_frenk_train, frenk_label_train, epochs=5, batch_size=32)

    probs = hate_model.predict(conc_frenk_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 2s 2ms/step - loss: 0.5033 - accuracy: 0.7467
Epoch 2/5
263/263 [==============================] - 1s 2ms/step - loss: 0.4680 - accuracy: 0.7686
Epoch 3/5
263/263 [==============================] - 1s 2ms/step - loss: 0.4578 - accuracy: 0.7770
Epoch 4/5
263/263 [==============================] - 1s 2ms/step - loss: 0.4405 - accuracy: 0.7811
Epoch 5/5
72/72 [==============================] - 0s 1ms/step


In [26]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8055687767729652
Avg. f1 score: 0.702909090909091


In [27]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.7601043024771839
Avg. f1 score: 0.6429495472186287


In [28]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.7792264232942199
Avg. f1 score: 0.40930232558139534


In [29]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.6588439808778792
Avg. f1 score: 0.5844362096347273


In [30]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

1/1 [==============================] - 0s 21ms/step
Hate prob.: [[0.20094016]]
1/1 [==============================] - 0s 31ms/step
Hate prob.: [[0.54898757]]


### Training with Gab

In [49]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_ghc_train, ghc_label_train, epochs=5, batch_size=32)

    probs = hate_model.predict(conc_ghc_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 1ms/step - loss: 0.2664 - accuracy: 0.8933
Epoch 2/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2495 - accuracy: 0.8989
Epoch 3/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2387 - accuracy: 0.8992
Epoch 4/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2345 - accuracy: 0.9033
Epoch 5/5
72/72 [==============================] - 0s 1ms/step


In [50]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.9079009995240361
Avg. f1 score: 0.47702702702702704


In [51]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.8900478053020426
Avg. f1 score: 0.38442822384428227


In [52]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6544980443285529
Avg. f1 score: 0.19289340101522842


In [53]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.7409821816601477
Avg. f1 score: 0.5730659025787965


In [54]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

1/1 [==============================] - 0s 22ms/step
Hate prob.: [[0.04365391]]
1/1 [==============================] - 0s 21ms/step
Hate prob.: [[0.05601781]]


### Training with Twi-Red-You

In [62]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    hate_model = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model.fit(conc_try_train, try_label_train, epochs=5, batch_size=32)

    probs = hate_model.predict(conc_try_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = hate_model.predict(conc_try_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))

    probs = hate_model.predict(conc_frenk_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model.predict(conc_ghc_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 2ms/step - loss: 0.4255 - accuracy: 0.7947
Epoch 2/5
263/263 [==============================] - 1s 2ms/step - loss: 0.3969 - accuracy: 0.8094
Epoch 3/5
263/263 [==============================] - 1s 2ms/step - loss: 0.3858 - accuracy: 0.8188
Epoch 4/5
263/263 [==============================] - 1s 2ms/step - loss: 0.3802 - accuracy: 0.8188
Epoch 5/5
72/72 [==============================] - 0s 1ms/step


In [63]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8309138505473584
Avg. f1 score: 0.7087517934002869


In [64]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.803129074315515
Avg. f1 score: 0.6681318681318681


In [65]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6671012603215993
Avg. f1 score: 0.2800751879699248


In [66]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.8613646240764885
Avg. f1 score: 0.3312368972746331


In [67]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_PEACE(text,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_PEACE(text_aggr,hate_model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

1/1 [==============================] - 0s 34ms/step
Hate prob.: [[0.09092075]]
1/1 [==============================] - 0s 18ms/step
Hate prob.: [[0.12210806]]


## Hate prediction with the general model

In [68]:
##Counterfactual evaluation
def evaluate_text_gen(text,model,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size):
    _,_,out_gen = Compute_Embeddings(tokenizer,sent_mod,aggr_mod,gen_mod,text,batch_size)
    prob = model.predict(out_gen)
    print("Hate prob.:",prob)

### Training with FRENK

In [69]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(frenk_gen_train, frenk_label_train, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(frenk_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 1ms/step - loss: 0.5674 - accuracy: 0.7011
Epoch 2/5
263/263 [==============================] - 0s 1ms/step - loss: 0.5066 - accuracy: 0.7418
Epoch 3/5
263/263 [==============================] - 0s 1ms/step - loss: 0.4863 - accuracy: 0.7540
Epoch 4/5
263/263 [==============================] - 0s 1ms/step - loss: 0.4692 - accuracy: 0.7607
Epoch 5/5
72/72 [==============================] - 0s 857us/step


In [70]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8029509757258448
Avg. f1 score: 0.7233544938189109


In [71]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.7518470230334637
Avg. f1 score: 0.6662770309760373


In [72]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.6688396349413298
Avg. f1 score: 0.34196891191709844


In [73]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.6149500217296827
Avg. f1 score: 0.550709939148073


In [74]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

1/1 [==============================] - 0s 19ms/step
Hate prob.: [[0.29789346]]
1/1 [==============================] - 0s 19ms/step
Hate prob.: [[0.61289006]]


### Training with Gab

In [75]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(ghc_gen_train, ghc_label_train, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(ghc_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 1ms/step - loss: 0.3295 - accuracy: 0.8772
Epoch 2/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2783 - accuracy: 0.8922
Epoch 3/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2645 - accuracy: 0.8974
Epoch 4/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2585 - accuracy: 0.8965
Epoch 5/5
72/72 [==============================] - 0s 870us/step


In [76]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8956449309852451
Avg. f1 score: 0.530765115034778


In [77]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.8843980877879183
Avg. f1 score: 0.4864864864864864


In [78]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6753585397653195
Avg. f1 score: 0.36962025316455693


In [79]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.6936114732724902
Avg. f1 score: 0.578096947935368


In [80]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

1/1 [==============================] - 0s 19ms/step
Hate prob.: [[0.04720673]]
1/1 [==============================] - 0s 25ms/step
Hate prob.: [[0.46948156]]


### Training with Twi-Red-You

In [81]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    hate_model_gen = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    hate_model_gen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    hate_model_gen.fit(try_gen_train, try_label_train, epochs=5, batch_size=32)

    probs = hate_model_gen.predict(try_gen_train)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = hate_model_gen.predict(try_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))
    
    probs = hate_model_gen.predict(frenk_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))

    probs = hate_model_gen.predict(ghc_gen_test)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 1ms/step - loss: 0.4626 - accuracy: 0.7734
Epoch 2/5
263/263 [==============================] - 0s 1ms/step - loss: 0.4132 - accuracy: 0.8001
Epoch 3/5
263/263 [==============================] - 0s 1ms/step - loss: 0.4033 - accuracy: 0.8100
Epoch 4/5
263/263 [==============================] - 0s 2ms/step - loss: 0.3933 - accuracy: 0.8149
Epoch 5/5
72/72 [==============================] - 0s 1ms/step


In [82]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8371013802950976
Avg. f1 score: 0.6847801059175684


In [83]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.7892220773576706
Avg. f1 score: 0.6008230452674896


In [84]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6471099521946979
Avg. f1 score: 0.2054794520547945


In [85]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.8587570621468926
Avg. f1 score: 0.2663656884875847


In [86]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_gen(text,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)
evaluate_text_gen(text_aggr,hate_model_gen,tokenizer,sent_mod,aggr_mod,gen_mod,batch_size)

1/1 [==============================] - 0s 18ms/step
Hate prob.: [[0.17148471]]
1/1 [==============================] - 0s 19ms/step
Hate prob.: [[0.5146115]]


## HateBERT

In [87]:
model_dir = "HateBERT_hateval"
HateBERT = BertModel.from_pretrained(
    model_dir # Use pre-trained model from its directory, change this to use a pre-trained model from bert
)
HateBERT.eval()
HateBERT_tok = BertTokenizer.from_pretrained(model_dir)

Some weights of the model checkpoint at HateBERT_hateval were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [88]:
def Comp_Embed_HateBERT(tokenizer,mod,texts,batch_size):
  # Split texts into batches
  batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
  batch_out = []

  with torch.no_grad():
      for batch in batches:
          encoded_inputs = tokenizer(batch,padding=True,return_tensors='pt')
          # Compute the output of the module as a numpy array.

          emb = mod(**encoded_inputs)[0]
          tf_emb = tf.convert_to_tensor(emb.numpy())  
          pool = GlobalAveragePooling1D()(tf_emb).numpy()
          batch_out.append(pool)

          encoded_inputs = None
          tf_emb = None
          gc.collect() 
    
  outs = np.concatenate(batch_out,axis=0)
  return outs

## Compute HateBERT embeddings (If these outputs are already saved you can skip this step)

### FRENK dataset

In [57]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,frenk_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/frenk/HateBERT_train_emb.npy',outs_train)

Elapsed time: 482.1618766784668
(8404, 768)


In [58]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,frenk_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/frenk/HateBERT_test_emb.npy',outs_test)

Elapsed time: 129.07557344436646
(2301, 768)


### Gab dataset

In [59]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,ghc_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/ghc/HateBERT_train_emb.npy',outs_train)

Elapsed time: 1384.8847632408142
(22036, 768)


In [60]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,ghc_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/ghc/HateBERT_test_emb.npy',outs_test)

Elapsed time: 339.3550543785095
(5510, 768)


### Twi-Red-You dataset

In [61]:
###Train set
batch_size = 32

t = time()
outs_train = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,try_train,batch_size)
print("Elapsed time:",time()-t)
print(outs_train.shape)

np.save('outs/try/HateBERT_train_emb.npy',outs_train)

Elapsed time: 464.42991185188293
(8404, 768)


In [62]:
###Test set
t = time()
outs_test = Comp_Embed_HateBERT(HateBERT_tok,HateBERT,try_test,batch_size)
print("Elapsed time:",time()-t)
print(outs_test.shape)

np.save('outs/try/HateBERT_test_emb.npy',outs_test)

Elapsed time: 125.01945090293884
(2301, 768)


## Hate prediction with HateBERT

In [89]:
frenk_train_emb = np.load('outs/frenk/HateBERT_train_emb.npy')
frenk_test_emb = np.load('outs/frenk/HateBERT_test_emb.npy')

In [90]:
ghc_train_emb = np.load('outs/ghc/HateBERT_train_emb.npy')
ghc_test_emb = np.load('outs/ghc/HateBERT_test_emb.npy')

In [91]:
ghc_train_emb = ghc_train_emb[:8404]
ghc_test_emb = ghc_test_emb[:2301]

In [92]:
try_train_emb = np.load('outs/try/HateBERT_train_emb.npy')
try_test_emb = np.load('outs/try/HateBERT_test_emb.npy')

In [93]:
##Counterfactual evaluation
def evaluate_text_HateBERT(text,model,tokenizer,HateBERT,batch_size):
    out = Comp_Embed_HateBERT(tokenizer,HateBERT,text,batch_size)
    prob = model.predict(out)
    print("Hate prob.:",prob)

### Training with FRENK

In [94]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_ghc = []
f1_test_ghc = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(frenk_train_emb, frenk_label_train, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(frenk_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,frenk_label_train))
    f1_train.append(metrics.f1_score(preds,frenk_label_train,zero_division=1))

    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 1ms/step - loss: 0.5276 - accuracy: 0.7285
Epoch 2/5
263/263 [==============================] - 0s 1ms/step - loss: 0.4696 - accuracy: 0.7728
Epoch 3/5
263/263 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.7905
Epoch 4/5
263/263 [==============================] - 0s 1ms/step - loss: 0.3927 - accuracy: 0.8154
Epoch 5/5
72/72 [==============================] - 0s 957us/step


In [95]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8907663017610662
Avg. f1 score: 0.8492610837438423


In [96]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

FRENK Test set:
Avg. accuracy: 0.7483702737940026
Avg. f1 score: 0.6596119929453262


In [97]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.7296827466318991
Avg. f1 score: 0.376753507014028


In [98]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.5949587136027814
Avg. f1 score: 0.5390702274975272


In [99]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

1/1 [==============================] - 0s 19ms/step
Hate prob.: [[0.4193343]]
1/1 [==============================] - 0s 40ms/step
Hate prob.: [[0.6952586]]


### Training with Gab

In [100]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_try = []
f1_test_try = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(ghc_train_emb, ghc_label_train, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(ghc_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,ghc_label_train))
    f1_train.append(metrics.f1_score(preds,ghc_label_train,zero_division=1))

    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_try.append(metrics.accuracy_score(preds,try_label_test))
    f1_test_try.append(metrics.f1_score(preds,try_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 1ms/step - loss: 0.2991 - accuracy: 0.8846
Epoch 2/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2682 - accuracy: 0.8937
Epoch 3/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2543 - accuracy: 0.8996
Epoch 4/5
263/263 [==============================] - 0s 1ms/step - loss: 0.2420 - accuracy: 0.9055
Epoch 5/5
72/72 [==============================] - 0s 933us/step


In [101]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.920871013802951
Avg. f1 score: 0.5991561181434599


In [102]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Gab Test set:
Avg. accuracy: 0.8813559322033898
Avg. f1 score: 0.3973509933774834


In [103]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6610169491525424
Avg. f1 score: 0.27777777777777773


In [104]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test_try))
print("Avg. f1 score:",np.mean(f1_test_try))

Twi-Red-You Test set:
Avg. accuracy: 0.7092568448500652
Avg. f1 score: 0.5773847125710676


In [105]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

1/1 [==============================] - 0s 40ms/step
Hate prob.: [[0.02501187]]
1/1 [==============================] - 0s 19ms/step
Hate prob.: [[0.40586358]]


### Training with Twi-Red-You 

In [93]:
acc_train = []
f1_train = []
acc_test = []
f1_test = []
acc_test_frenk = []
f1_test_frenk = []
acc_test_ghc = []
f1_test_ghc = []
for i in range(n_avg):
    classifier_HateBERT = Sequential([
          Flatten(),
          Dense(128, activation='relu'),
          Dense(128, activation='relu'),
          Dense(1, activation='sigmoid')
      ])
    classifier_HateBERT.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    classifier_HateBERT.fit(try_train_emb, try_label_train, epochs=5, batch_size=32)

    probs = classifier_HateBERT.predict(try_train_emb)
    preds = (probs >= 0.5).astype(int)
    acc_train.append(metrics.accuracy_score(preds,try_label_train))
    f1_train.append(metrics.f1_score(preds,try_label_train,zero_division=1))
    
    probs = classifier_HateBERT.predict(try_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test.append(metrics.accuracy_score(preds,try_label_test))
    f1_test.append(metrics.f1_score(preds,try_label_test,zero_division=1))
    
    probs = classifier_HateBERT.predict(frenk_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_frenk.append(metrics.accuracy_score(preds,frenk_label_test))
    f1_test_frenk.append(metrics.f1_score(preds,frenk_label_test,zero_division=1))

    probs = classifier_HateBERT.predict(ghc_test_emb)
    preds = (probs >= 0.5).astype(int)
    acc_test_ghc.append(metrics.accuracy_score(preds,ghc_label_test))
    f1_test_ghc.append(metrics.f1_score(preds,ghc_label_test,zero_division=1))

Epoch 1/5
263/263 [==============================] - 1s 1ms/step - loss: 0.4303 - accuracy: 0.7951
Epoch 2/5
263/263 [==============================] - 0s 1ms/step - loss: 0.3885 - accuracy: 0.8187
Epoch 3/5
263/263 [==============================] - 0s 1ms/step - loss: 0.3636 - accuracy: 0.8326
Epoch 4/5
263/263 [==============================] - 0s 1ms/step - loss: 0.3317 - accuracy: 0.8460
Epoch 5/5
72/72 [==============================] - 0s 987us/step


In [94]:
print("Train set:")
print("Avg. accuracy:",np.mean(acc_train))
print("Avg. f1 score:",np.mean(f1_train))

Train set:
Avg. accuracy: 0.8830318895763922
Avg. f1 score: 0.8000813504169209


In [95]:
print("Twi-Red-You Test set:")
print("Avg. accuracy:",np.mean(acc_test))
print("Avg. f1 score:",np.mean(f1_test))

Twi-Red-You Test set:
Avg. accuracy: 0.782703172533681
Avg. f1 score: 0.6463932107496463


In [96]:
print("FRENK Test set:")
print("Avg. accuracy:",np.mean(acc_test_frenk))
print("Avg. f1 score:",np.mean(f1_test_frenk))

FRENK Test set:
Avg. accuracy: 0.6584093872229465
Avg. f1 score: 0.25426944971537


In [97]:
print("Gab Test set:")
print("Avg. accuracy:",np.mean(acc_test_ghc))
print("Avg. f1 score:",np.mean(f1_test_ghc))

Gab Test set:
Avg. accuracy: 0.8461538461538461
Avg. f1 score: 0.3244274809160305


In [98]:
text = ["We are ALL tired of waiting . . realize something"]
text_aggr = ["We’ve had ENOUGH of your delays. Get your act together NOW"]

evaluate_text_HateBERT(text,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)
evaluate_text_HateBERT(text_aggr,classifier_HateBERT,HateBERT_tok,HateBERT,batch_size)

1/1 [==============================] - 0s 19ms/step
Hate prob.: [[0.00607339]]
1/1 [==============================] - 0s 20ms/step
Hate prob.: [[0.40812752]]
